# 작사가 인공지능 만들기

## 필요한 라이브러리 import 하기

In [28]:
import glob
import os
import re
import numpy as np
import tensorflow as tf
import itertools

## 데이터 읽어오기

In [29]:
txt_file_path = os.getenv('HOME')+'/aiffel/Ex_4/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 중복 파일 제거
# https://aiffel.agit.io/g/300308438/wall/327378410
def check_dup(file1, file2):
    txt1 = []
    txt2 = []
    with open(file1, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt1.extend(raw)
    with open(file2, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt2.extend(raw)
    txt1 = set(txt1)
    txt2 = set(txt2)
    diff = txt1.difference(txt2)
    return len(txt1) * 0.05 > len(diff)


for a, b in itertools.combinations(txt_list, 2):
    if check_dup(a, b):
        print(a, b)
        txt_list.remove(b)

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print(len(raw_corpus))  # 중복 파일 제거 후 의 corpus -> 원본 데이터 크기 : 187088

/aiffel/aiffel/Ex_4/data/lyrics/notorious_big.txt /aiffel/aiffel/Ex_4/data/lyrics/notorious-big.txt
/aiffel/aiffel/Ex_4/data/lyrics/Kanye_West.txt /aiffel/aiffel/Ex_4/data/lyrics/kanye-west.txt
175613


In [30]:
print(raw_corpus[:9])  # 앞에서부터 10라인 출력.

["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah']


## 데이터 정제하기

In [31]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                    # 1.소문자로 바꾸고, 양쪽 공백을 지웁니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 2. 특수문자 양쪽에 공백을 넣고.
    sentence = re.sub(r'[" "]+', " ", sentence)            # 3. 여러개의 공백은 하나의 공백으로 바꿉니다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다.
    sentence = sentence.strip()                            # 5. 다시 양쪽 공백을 지웁니다.
    sentence = '<start> ' + sentence + ' <end>'            # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다.
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))  # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [32]:
# 여기에 정제된 문장을 모을겁니다.
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다.
    if len(sentence) == 0 : continue
    if sentence[-1] == ":" : continue
    if sentence[0] == "[" : continue
    if sentence.count(" ") > 10 : continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인하기.
print(corpus[:10])
# 데이터 확인
print()
print(len(corpus))

['<start> now i ve heard there was a secret chord <end>', '<start> that david played , and it pleased the lord <end>', '<start> but you don t really care for music , do you ? <end>', '<start> it goes like this <end>', '<start> the fourth , the fifth <end>', '<start> the minor fall , the major lift <end>', '<start> the baffled king composing hallelujah hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah your faith was strong but you needed proof <end>']

140658


In [33]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다.
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000,   # 20000단어를 기억할 수 있는 tokenizer를 만들겁니다.
        filters=' ',       # 우리는 이미 문장을 정제했으니 filters가 필요없어요.
        oov_token="<unk>"  # 20000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요.
    )
    tokenizer.fit_on_texts(corpus)  # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다.
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)  # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다.
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   53    5 ...    0    0    0]
 [   2   17 3054 ...    0    0    0]
 [   2   36    7 ...   39    3    0]
 ...
 [   2    5  117 ...    0    0    0]
 [   2  275  194 ...   12    3    0]
 [   2    7   40 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fb9d8489550>


In [34]:
print(tensor.shape)

(140658, 15)


In [35]:
print(tensor[:3, :10])  # 텐서 데이터의 3번째 행, 10번째 열.

[[   2   53    5   86  325   62   59    9  938 5667]
 [   2   17 3054  989    4    8   11 5668    6  334]
 [   2   36    7   30   14  159  283   28  307    4]]


In [36]:
print(tensor[:3, :])

[[   2   53    5   86  325   62   59    9  938 5667    3    0    0    0
     0]
 [   2   17 3054  989    4    8   11 5668    6  334    3    0    0    0
     0]
 [   2   36    7   30   14  159  283   28  307    4   47    7   39    3
     0]]


In [37]:
print(tensor[:10, :])

[[    2    53     5    86   325    62    59     9   938  5667     3     0
      0     0     0]
 [    2    17  3054   989     4     8    11  5668     6   334     3     0
      0     0     0]
 [    2    36     7    30    14   159   283    28   307     4    47     7
     39     3     0]
 [    2    11   308    25    43     3     0     0     0     0     0     0
      0     0     0]
 [    2     6  4049     4     6  2122     3     0     0     0     0     0
      0     0     0]
 [    2     6  6193   285     4     6  1120   718     3     0     0     0
      0     0     0]
 [    2     6  7808   508 11283   889   889     3     0     0     0     0
      0     0     0]
 [    2   889     3     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    2   889     3     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    2   889    21   812    59   551    36     7   950  1616     3     0
      0     0     0]]


In [38]:
# 텐서 데이터는 모두 정수 -> tokenizer에 구축된 단어 사전의 인덱스.
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [39]:
print(len(tensor[0]))

15


In [40]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
short_tensor = np.array([x for i, x in enumerate(tensor) if x[-1]==0 or x[-1]==3])  # 따라서 마지막 토큰이 0이거나 3인것만 선택.
print(short_tensor.shape)
src_input = short_tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    
    
print(src_input[0])
print(tgt_input[0])

(140658, 15)
[   2   53    5   86  325   62   59    9  938 5667    3    0    0    0]
[  53    5   86  325   62   59    9  938 5667    3    0    0    0    0]


## 평가 데이터셋 분리하기

In [41]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         shuffle=True,
                                                         random_state=42)
print(len(enc_train), len(enc_val))
print(enc_train.shape)
print(dec_train.shape)

112526 28132
(112526, 14)
(112526, 14)


## 훈련데이터셋 생성하기

In [42]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개.
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다.
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## TextGenerator 모델 생성하기

In [43]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [44]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다.
model(src_sample)

<tf.Tensor: shape=(256, 14, 20001), dtype=float32, numpy=
array([[[-1.02550883e-04, -2.63967348e-04,  8.98487851e-05, ...,
         -2.02919240e-04,  1.16412222e-04,  3.79250341e-05],
        [-4.89040278e-04, -1.99378221e-04, -1.39098440e-04, ...,
         -5.41711168e-04,  3.28094815e-04, -2.63815127e-05],
        [-1.03702839e-03,  2.73560745e-05, -5.55323844e-04, ...,
         -9.03519744e-04,  5.70681412e-04, -1.55132904e-04],
        ...,
        [-2.80114851e-04, -4.05031096e-05, -1.45437277e-03, ...,
         -1.34142954e-03, -1.79814422e-04,  1.90009800e-04],
        [-1.06738415e-04, -1.17518990e-04, -1.51244283e-03, ...,
         -1.51242968e-03, -2.73846847e-04,  3.26090172e-04],
        [ 3.17694285e-05, -1.84139775e-04, -1.58108643e-03, ...,
         -1.65881403e-03, -3.39443359e-04,  4.27707360e-04]],

       [[-1.02550883e-04, -2.63967348e-04,  8.98487851e-05, ...,
         -2.02919240e-04,  1.16412222e-04,  3.79250341e-05],
        [-2.10944097e-04, -2.77278334e-04,  1

In [45]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  5120256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  20501025  
Total params: 39,260,961
Trainable params: 39,260,961
Non-trainable params: 0
_________________________________________________________________


## 모델 훈련하기

In [46]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
439/439 [==============================] - 101s 226ms/step - loss: 3.5632
Epoch 2/10
439/439 [==============================] - 100s 227ms/step - loss: 3.0338
Epoch 3/10
439/439 [==============================] - 100s 228ms/step - loss: 2.8775
Epoch 4/10
439/439 [==============================] - 100s 228ms/step - loss: 2.7639
Epoch 5/10
439/439 [==============================] - 100s 227ms/step - loss: 2.6674
Epoch 6/10
439/439 [==============================] - 100s 227ms/step - loss: 2.5814
Epoch 7/10
439/439 [==============================] - 100s 228ms/step - loss: 2.5027
Epoch 8/10
439/439 [==============================] - 100s 227ms/step - loss: 2.4296
Epoch 9/10
439/439 [==============================] - 100s 228ms/step - loss: 2.3615
Epoch 10/10
439/439 [==============================] - 100s 227ms/step - loss: 2.2975


## 추가 재학습

In [47]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개.
VOCAB_SIZE = tokenizer.num_words + 1   

dataset_128 = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_128 = dataset_128.shuffle(BUFFER_SIZE)
dataset_128 = dataset_128.batch(BATCH_SIZE, drop_remainder=True)
dataset_128

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

In [48]:
embedding_size = 256 + 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_128, epochs=10)

Epoch 1/10
879/879 [==============================] - 111s 122ms/step - loss: 3.2210
Epoch 2/10
879/879 [==============================] - 107s 122ms/step - loss: 2.7787
Epoch 3/10
879/879 [==============================] - 107s 122ms/step - loss: 2.5849
Epoch 4/10
879/879 [==============================] - 107s 122ms/step - loss: 2.4082
Epoch 5/10
879/879 [==============================] - 107s 122ms/step - loss: 2.2427
Epoch 6/10
879/879 [==============================] - 107s 122ms/step - loss: 2.0925
Epoch 7/10
879/879 [==============================] - 107s 122ms/step - loss: 1.9581
Epoch 8/10
879/879 [==============================] - 107s 122ms/step - loss: 1.8369
Epoch 9/10
879/879 [==============================] - 107s 122ms/step - loss: 1.7274
Epoch 10/10
879/879 [==============================] - 107s 122ms/step - loss: 1.6263


## 학습 가중치 저장하기

In [49]:
model.save_weights(os.getenv('HOME') + '/aiffel/Ex_4/my_checkpoint')

In [51]:
embedding_size = 256 + 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model.load_weights(os.getenv('HOME') + '/aiffel/Ex_4/my_checkpoint')

## 평가 데이터셋 생성하기

In [54]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256 + 128
steps_per_epoch = len(enc_val) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((384, 14), (384, 14)), types: (tf.int32, tf.int32)>

## 평가하기

In [55]:
lyricist = model
lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.evaluate(val_dataset)

73/73 [==============================] - 10s 131ms/step - loss: 2.5361


2.5361459255218506

## 문장 생성 해보기

In [56]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [57]:
generate_text(lyricist, tokenizer, init_sentence="<start> there")

'<start> there s a creak behind the door <end> '

In [58]:
generate_text(lyricist, tokenizer, init_sentence="<start> you")

'<start> you re the only one i want <end> '

In [59]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love")

'<start> i love you <end> '

In [60]:
generate_text(lyricist, tokenizer, init_sentence="<start> True")

'<start> true love needs a lover <end> '

In [61]:
generate_text(lyricist, tokenizer, init_sentence="<start> say")

'<start> say it all , <end> '

In [66]:
generate_text(lyricist, tokenizer, init_sentence="<start> when we' re")

'<start> when <unk> re the only felony <end> '

# 회고

NLP 첫 익스라서 나름 관심갖고 해보려 했는데 노드밀려서 시간을 생각보다 못썼고 학습시간이 길어서 많이 실험을 해보지 못했다..ㅠ
더 좋은 결과를 위해 이거저거 코드도 참고해보고 해보았지만 val_loss를 2.2 이하로 낮추지는 못했다...
아쉬운 결과지만 많이 배웠고 또 시간나면 더 해봐야 겠다.
문장은 나름 잘 생성하였고 ' 처리를 좀 신경써야겠다.
정규표현식 공부!